In [253]:
import torch
import json
import datasets

from datasets import Dataset, load_dataset, Value, Features
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, Trainer, TrainingArguments, DataCollatorWithPadding, DefaultDataCollator, pipeline

# Загрузка языковой модели и токенизатора
model_name = 'timpal0l/mdeberta-v3-base-squad2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

In [230]:
with open(r'dataset\train\train.json', 'r', encoding='utf-8') as fcc_file:
    train_json = json.load(fcc_file)


train_data = []

for i in range(len(train_json)):
    train_data.append(train_json[str(i)])
# train_data = Dataset()

In [231]:
with open(r'dataset\test\test.json', 'r', encoding='utf-8') as fcc_file:
    test_json = json.load(fcc_file)


test_data = []

for i in range(len(train_json)):
    try:
        test_data.append(test_json[str(len(train_json)+i)])
    except:
        pass

In [232]:
test_data[-1]

{'answers': {'answer_start': [49],
  'text': ['Погрешность датчика терморегулятора составляет 5-7 С, что не позволяет точно установить температуру в пределах 180-220С Что бы исправить Корректировка диапазонов температур по карте режимов наладки на 190-210 С']},
 'context': 'Нагрев материала в шнеке ТПА более чем на 220 С Погрешность датчика терморегулятора составляет 5-7 С, что не позволяет точно установить температуру в пределах 180-220С Что бы исправить Корректировка диапазонов температур по карте режимов наладки на 190-210 С',
 'id': '546',
 'question': 'Возникла проблема: нагрев материала в шнеке тпа более чем на 220 с',
 'title': 'factory'}

In [233]:
new_dataset = datasets.DatasetDict({"train":Dataset.from_list(train_data),"test":Dataset.from_list(test_data)})
new_dataset['train'][0]
#train_data[0]

{'answers': {'answer_start': [40],
  'text': ['Нарушено правило изоляции/идентификации наладочного брака Что бы исправить Ревизия рабочего места, эргономика рабочего места']},
 'context': 'Пересорт (попадание наладочного брака) Нарушено правило изоляции/идентификации наладочного брака Что бы исправить Ревизия рабочего места, эргономика рабочего места',
 'id': '0',
 'question': 'Возникла проблема: пересорт (попадание наладочного брака)',
 'title': 'factory'}

In [234]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [235]:
tokenized_new_dataset = new_dataset.map(preprocess_function, batched=True, remove_columns=new_dataset["train"].column_names)

Map:   0%|          | 0/383 [00:00<?, ? examples/s]

Map:   0%|          | 0/164 [00:00<?, ? examples/s]

In [236]:
data_collator = DefaultDataCollator()

In [237]:
training_args = TrainingArguments(model_name)

In [247]:
training_args = TrainingArguments(
    output_dir=r"models/weights/",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
)


In [248]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_new_dataset["train"],
    eval_dataset=tokenized_new_dataset["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
)

In [249]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: ERROR API key must be 40 characters long, yours was 37
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: ERROR API key must be 40 characters long, yours was 37
wandb: Logging

  0%|          | 0/72 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

{'eval_loss': 0.27453091740608215, 'eval_runtime': 55.5039, 'eval_samples_per_second': 2.955, 'eval_steps_per_second': 0.198, 'epoch': 1.0}


  0%|          | 0/11 [00:00<?, ?it/s]

{'eval_loss': 0.1808600127696991, 'eval_runtime': 53.979, 'eval_samples_per_second': 3.038, 'eval_steps_per_second': 0.204, 'epoch': 2.0}


  0%|          | 0/11 [00:00<?, ?it/s]

{'eval_loss': 0.21363955736160278, 'eval_runtime': 56.6892, 'eval_samples_per_second': 2.893, 'eval_steps_per_second': 0.194, 'epoch': 3.0}
{'train_runtime': 1374.5788, 'train_samples_per_second': 0.836, 'train_steps_per_second': 0.052, 'train_loss': 0.4771087434556749, 'epoch': 3.0}


TrainOutput(global_step=72, training_loss=0.4771087434556749, metrics={'train_runtime': 1374.5788, 'train_samples_per_second': 0.836, 'train_steps_per_second': 0.052, 'total_flos': 225176546363904.0, 'train_loss': 0.4771087434556749, 'epoch': 3.0})

In [ ]:
trainer.push_to_hub()

In [260]:
question = "Возникла проблема: Не правильно спроектирована пресс-форма"
# context = "Не правильно спроектирована пресс-форма Что бы исправить Подобраны оптимальные параметры нагрева материала в печи. Введены допуска в карту режимов на параметры температуры и времени нагрева."

In [257]:
question_answerer = pipeline("question-answering", model=r"models\weights\checkpoint-72")
# question_answerer(question=question, context=context)

Device set to use cuda:0


{'score': 0.17453309893608093, 'start': 56, 'end': 66, 'answer': ' Подобраны'}

In [261]:
with open(r"models\new_context.txt", "r", encoding="utf8") as file:
        context = file.read()
        
question_answerer = pipeline("question-answering", model=r"models\weights\checkpoint-72")
question_answerer(question=question, context=context)

Device set to use cuda:0


{'score': 0.3597894608974457,
 'start': 29039,
 'end': 29055,
 'answer': '\n2.необученность'}